In [2]:
import os
import pandas as pd
import numpy as np
from collections import Counter
import copy

from sklearn.utils import resample

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore",category=UserWarning, append=True)

# from scikit-beam.utils.exceptions import Scikit-beamWarningg
# warnings.simplefilter('ignore', category=Scikit-beamWarning)

import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

## with balanced dataset

In [3]:
def cv_tune_bl_split(dat_use, # dataframe with at leat one column name: outcome, one column name: ID
                     outcome, # outcome column
                     ID,# sample ID column
                     var_, # selected variables
                     key,# string in the list rf, knn, logit_l1, logit_l2, elasticnet, mlp, xgb, mlp and svm
                     clf,
                     grid,
                     scoring='roc_auc', 
                     output_prefix = 'cv_tune', 
                     n_jobs=1, 
                     random_state=123, 
                     folder_out='/results/', 
                     kf_inner=5,
                     perc_train=0.8):
    
    samples_1 = dat_use.loc[dat_use[outcome] == 1, ID].values
    print(f'sample size group 1: {samples_1.shape}')
    

    samples_0 = dat_use.loc[dat_use[outcome] == 0, ID].values
    print(f'sample size group 0: {samples_0.shape}')
    
    n_train = math.ceil(min(len(samples_0), len(samples_1))*perc_train)

    print(key)
    eval_metrics = pd.DataFrame()

    for i in range(100):
        # samples of group 1 in test dataset
        samples_1_train = resample(samples_1, replace=(False), n_samples=n_train, random_state=(i))
        # samples_1_test = resample(samples_1, replace=(False), n_samples=n_test, random_state=(i))
        samples_1_test = samples_1[np.isin(samples_1, samples_1_train, invert=True)]

        # samples of group 0, the same number of training and test as in for group 1 from group 0
        samples_0_train = resample(samples_0, replace=(False), n_samples=n_train, random_state=(i))
        # samples_0_test = resample(samples_0, replace=(False), n_samples=n_test, random_state=(i))
        samples_0_ = samples_0[np.isin(samples_0, samples_0_train, invert=True)]
        samples_0_test = resample(samples_0_, replace=(False), n_samples=len(samples_1_test), random_state=(i))

        ##### 
        X_train = dat_use.loc[np.append(samples_0_train, samples_1_train), var_].values
        print(X_train.shape)
        y_train = dat_use.loc[np.append(samples_0_train, samples_1_train), outcome].values
        print(Counter(y_train))

        X_test = dat_use.loc[np.append(samples_0_test, samples_1_test), var_].values
        print(X_test.shape)
        y_test = dat_use.loc[np.append(samples_0_test, samples_1_test), outcome].values
        print(Counter(y_test))

        pipeline = Pipeline([('scale', PowerTransformer()), #####
                                # ("smote", SMOTE(random_state=random_state, n_jobs=n_jobs)), 
                              ("clf", clf)])

        # search = RandomizedSearchCV(
        #     pipeline, grid, 
        #     # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
        #     scoring=scoring,
        #     n_iter=100,random_state=random_state,
        #     n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        # ).fit(X_train, y_train)

        search = GridSearchCV(
            pipeline, grid, 
            # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
            scoring=scoring, 
            n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        ).fit(X_train, y_train)

        best_score = search.best_score_
        # print(f"Best Tuning balanced accuracy: {best_score}") ###############

        best_params = {
            key: value for key, value in search.best_params_.items()
        }
        # print(best_params)

        best_estimator = search.best_estimator_

        df_proba = pd.DataFrame()

        ave = 'macro'

        # y_pred_cv = np.append(y_pred_cv, y_pred)

        # area under ROC curve
        y_probs = best_estimator.predict_proba(X_test)[:,1]
        # y_prob_cv = np.append(y_prob_cv, y_probs)

        fp_rate, tp_rate, thresholds = metrics.roc_curve(y_test, y_probs)
        auc = metrics.auc(fp_rate, tp_rate)
        # auc_weight = metrics.roc_auc_score(y_test, y_pred_weight)

        y_pred = best_estimator.predict(X_test)

        # # probability
        # df_proba_ = pd.DataFrame()
        # df_proba_['proba'] = y_probs
        # df_proba_['true'] = y_test
        # df_proba = df_proba.append(df_proba_, ignore_index=True)

        # area under precision-recall curve
        precision_, recall_, thresholds_ = metrics.precision_recall_curve(y_test, y_probs)
        auc_pr = metrics.auc(recall_, precision_)

        precision = metrics.precision_score(y_test, y_pred, average=ave)
        precision_1 = metrics.precision_score(y_test, y_pred)
        precision_0 = metrics.precision_score(y_test, y_pred, pos_label=0)
        f1 = metrics.f1_score(y_test, y_pred, average=ave)
        f1_1 = metrics.f1_score(y_test, y_pred)
        f1_0 = metrics.f1_score(y_test, y_pred, pos_label=0)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        mcc = metrics.matthews_corrcoef(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred, average=ave)
        recall_1 = metrics.recall_score(y_test, y_pred)
        recall_0 = metrics.recall_score(y_test, y_pred, pos_label=0)
        balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

        brier = metrics.brier_score_loss(y_test, y_probs)

        eval_metrics = pd.concat([eval_metrics,pd.DataFrame.from_dict({'auroc': auc,
                                            'accuracy': accuracy,
                                            'sensitivity': recall_1,
                                            'specificity': recall_0,
                                            'brier': brier}, orient='index').transpose()], ignore_index=True)    
    file_out = output_prefix+key+'.csv' ###############
    eval_metrics.to_csv(os.path.join(folder_out,file_out), index=False)
    
    return eval_metrics
    
    

## with balanced dataset and split index

In [5]:
def cv_tune_bl_idx(dat_use,# dataframe with at leat one column name: outcome, one column name: ID
                   outcome, # outcome column
                   ID, # sample ID column
                   var_, # selected variables
                   key, # string in the list rf, knn, logit_l1, logit_l2, elasticnet, mlp, xgb, mlp and svm
                   train_test, # balanced train-test split index, 1=train, 0=test
                   scoring='roc_auc', 
                   output_prefix = 'cv_tune', 
                   n_jobs=1, 
                   random_state=123, 
                   folder_out='/results/', 
                   kf_inner=5):
    
    clf = lst_clf[key]
    
    samples_1 = dat_use.loc[dat_use[outcome] == 1, ID].values
    print(f'sample size group 1: {samples_1.shape}')

    samples_0 = dat_use.loc[dat_use[outcome] == 0, ID].values
    print(f'sample size group 0: {samples_0.shape}')

    print(key)
    eval_metrics = pd.DataFrame()

    for i in range(100):
        # train test index from outside
        samples_train = train_test.loc[train_test['train_'+str(i+1)] == 1,ID].values
        samples_test = train_test.loc[train_test['train_'+str(i+1)] == 0,ID].values ##### balanced test dataset or not
        # samples_train.shape
        # samples_test.shape

        samples_1_train = samples_train[np.isin(samples_train, samples_1)]
        samples_1_test = samples_test[np.isin(samples_test, samples_1)]

        samples_0_train = samples_train[np.isin(samples_train, samples_0)]
        samples_0_test = samples_test[np.isin(samples_test, samples_0)]

        ##### 
        X_train = dat_use.loc[np.append(samples_0_train, samples_1_train), var_].values
        print(X_train.shape)
        y_train = dat_use.loc[np.append(samples_0_train, samples_1_train), outcome].values
        print(Counter(y_train))

        X_test = dat_use.loc[np.append(samples_0_test, samples_1_test), var_].values
        print(X_test.shape)
        y_test = dat_use.loc[np.append(samples_0_test, samples_1_test), outcome].values
        print(Counter(y_test))

        pipeline = Pipeline([('scale', PowerTransformer()), #####
                                # ("smote", SMOTE(random_state=random_state, n_jobs=n_jobs)), 
                              ("clf", clf)])

        grid = lst_grid[key]

        # search = RandomizedSearchCV(
        #     pipeline, grid, 
        #     # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
        #     scoring=scoring,
        #     n_iter=100,random_state=random_state,
        #     n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        # ).fit(X_train, y_train)

        search = GridSearchCV(
            pipeline, grid, 
            # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
            scoring=scoring, 
            n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        ).fit(X_train, y_train)

        best_score = search.best_score_
        # print(f"Best Tuning balanced accuracy: {best_score}") ###############

        best_params = {
            key: value for key, value in search.best_params_.items()
        }
        # print(best_params)

        best_estimator = search.best_estimator_

        df_proba = pd.DataFrame()

        ave = 'macro'

        # y_pred_cv = np.append(y_pred_cv, y_pred)

        # area under ROC curve
        y_probs = best_estimator.predict_proba(X_test)[:,1]
        # y_prob_cv = np.append(y_prob_cv, y_probs)

        fp_rate, tp_rate, thresholds = metrics.roc_curve(y_test, y_probs)
        auc = metrics.auc(fp_rate, tp_rate)
        # auc_weight = metrics.roc_auc_score(y_test, y_pred_weight)

        y_pred = best_estimator.predict(X_test)

        # # probability
        # df_proba_ = pd.DataFrame()
        # df_proba_['proba'] = y_probs
        # df_proba_['true'] = y_test
        # df_proba = df_proba.append(df_proba_, ignore_index=True)

        # area under precision-recall curve
        precision_, recall_, thresholds_ = metrics.precision_recall_curve(y_test, y_probs)
        auc_pr = metrics.auc(recall_, precision_)

        precision = metrics.precision_score(y_test, y_pred, average=ave)
        precision_1 = metrics.precision_score(y_test, y_pred)
        precision_0 = metrics.precision_score(y_test, y_pred, pos_label=0)
        f1 = metrics.f1_score(y_test, y_pred, average=ave)
        f1_1 = metrics.f1_score(y_test, y_pred)
        f1_0 = metrics.f1_score(y_test, y_pred, pos_label=0)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        mcc = metrics.matthews_corrcoef(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred, average=ave)
        recall_1 = metrics.recall_score(y_test, y_pred)
        recall_0 = metrics.recall_score(y_test, y_pred, pos_label=0)
        balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

        brier = metrics.brier_score_loss(y_test, y_probs)

        eval_metrics = pd.concat([eval_metrics,pd.DataFrame.from_dict({'auroc': auc,
                                            'accuracy': accuracy,
                                            'sensitivity': recall_1,
                                            'specificity': recall_0,
                                            'brier': brier}, orient='index').transpose()], ignore_index=True)    
    file_out = output_prefix+key+'.csv' ###############
    eval_metrics.to_csv(os.path.join(folder_out,file_out), index=False)
    
    return eval_metrics

    # y_test_pred = pd.DataFrame(np.concatenate((y_test_cv.reshape((-1,1)), y_pred_cv.reshape((-1,1))), axis=1))
    # y_test_pred.columns = ['test','pred']
    # print(y_test_pred.shape)
    # pred_out = 'y_test_pred_cv5_'+key+'.csv' ###################
    # y_test_pred.to_csv(os.path.join(folder_out, pred_out), index=False)
    
    

## with balanced dataset and feature loop

In [ ]:
def cv_tune_bl_varLoop(dat_use, # dataframe with at leat one column name: outcome, one column name: ID
                       outcome, # outcome column
                       ID, # sample ID column
                       key, # string in the list rf, knn, logit_l1, logit_l2, elasticnet, mlp, xgb, mlp and svm
                       train_test, # balanced train-test split index, 1=train, 0=test
                       df_features, 
                       scoring='roc_auc', 
                       output_prefix = 'cv_tune', 
                       n_jobs=1, 
                       random_state=123, 
                       folder_out='/results/', 
                       kf_inner=5):

    print(key)
    
    clf = lst_clf[key]
    
    samples_1 = dat_use.loc[dat_use[outcome] == 1, ID].values
    print(f'sample size group 1: {samples_1.shape}')

    samples_0 = dat_use.loc[dat_use[outcome] == 0, ID].values
    print(f'sample size group 0: {samples_0.shape}')
    
    eval_metrics = pd.DataFrame()

    for i in range(100):
        # features selected in each iteration
        var_ = df_features.loc[df_features['select_'+str(i+1)] == 1, 'feature'].values
        print(var_.shape)

        # train test index from outside
        samples_train = train_test.loc[train_test['train_'+str(i+1)] == 1,ID].values
        samples_test = train_test.loc[train_test['train_'+str(i+1)] == 0,ID].values ##### balanced test dataset or not
        # samples_train.shape
        # samples_test.shape

        samples_1_train = samples_train[np.isin(samples_train, samples_1)]
        samples_1_test = samples_test[np.isin(samples_test, samples_1)]

        samples_0_train = samples_train[np.isin(samples_train, samples_0)]
        samples_0_test = samples_test[np.isin(samples_test, samples_0)]

        ##### 
        X_train = dat_use.loc[np.append(samples_0_train, samples_1_train), var_].values
        print(X_train.shape)
        y_train = dat_use.loc[np.append(samples_0_train, samples_1_train), outcome].values
        print(Counter(y_train))

        X_test = dat_use.loc[np.append(samples_0_test, samples_1_test), var_].values
        print(X_test.shape)
        y_test = dat_use.loc[np.append(samples_0_test, samples_1_test), outcome].values
        print(Counter(y_test))

        pipeline = Pipeline([('scale', PowerTransformer()), #####
                                # ("smote", SMOTE(random_state=random_state, n_jobs=n_jobs)), 
                              ("clf", clf)])

        grid = lst_grid[key]

        # search = RandomizedSearchCV(
        #     pipeline, grid, 
        #     # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
        #     scoring=scoring,
        #     n_iter=100,random_state=random_state,
        #     n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        # ).fit(X_train, y_train)

        search = GridSearchCV(
            pipeline, grid, 
            # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
            scoring=scoring, 
            n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        ).fit(X_train, y_train)

        best_score = search.best_score_
        # print(f"Best Tuning balanced accuracy: {best_score}") ###############

        best_params = {
            key: value for key, value in search.best_params_.items()
        }
        # print(best_params)

        best_estimator = search.best_estimator_

        df_proba = pd.DataFrame()

        ave = 'macro'

        # y_pred_cv = np.append(y_pred_cv, y_pred)

        # area under ROC curve
        y_probs = best_estimator.predict_proba(X_test)[:,1]
        # y_prob_cv = np.append(y_prob_cv, y_probs)

        fp_rate, tp_rate, thresholds = metrics.roc_curve(y_test, y_probs)
        auc = metrics.auc(fp_rate, tp_rate)
        # auc_weight = metrics.roc_auc_score(y_test, y_pred_weight)

        y_pred = best_estimator.predict(X_test)

        # # probability
        # df_proba_ = pd.DataFrame()
        # df_proba_['proba'] = y_probs
        # df_proba_['true'] = y_test
        # df_proba = df_proba.append(df_proba_, ignore_index=True)

        # area under precision-recall curve
        precision_, recall_, thresholds_ = metrics.precision_recall_curve(y_test, y_probs)
        auc_pr = metrics.auc(recall_, precision_)

        precision = metrics.precision_score(y_test, y_pred, average=ave)
        precision_1 = metrics.precision_score(y_test, y_pred)
        precision_0 = metrics.precision_score(y_test, y_pred, pos_label=0)
        f1 = metrics.f1_score(y_test, y_pred, average=ave)
        f1_1 = metrics.f1_score(y_test, y_pred)
        f1_0 = metrics.f1_score(y_test, y_pred, pos_label=0)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        mcc = metrics.matthews_corrcoef(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred, average=ave)
        recall_1 = metrics.recall_score(y_test, y_pred)
        recall_0 = metrics.recall_score(y_test, y_pred, pos_label=0)
        balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

        brier = metrics.brier_score_loss(y_test, y_probs)

        eval_metrics = pd.concat([eval_metrics,pd.DataFrame.from_dict({'auroc': auc,
                                            'accuracy': accuracy,
                                            'sensitivity': recall_1,
                                            'specificity': recall_0,
                                            'brier':brier}, orient='index').transpose()], ignore_index=True)    
    file_out = output_prefix+key+'.csv' ###############
    eval_metrics.to_csv(os.path.join(folder_out,file_out), index=False)
    
    return eval_metrics

    # y_test_pred = pd.DataFrame(np.concatenate((y_test_cv.reshape((-1,1)), y_pred_cv.reshape((-1,1))), axis=1))
    # y_test_pred.columns = ['test','pred']
    # print(y_test_pred.shape)
    # pred_out = 'y_test_pred_cv5_'+key+'.csv' ###################
    # y_test_pred.to_csv(os.path.join(folder_out, pred_out), index=False)
    
    

## imbalanced data

In [ ]:
def cv_tune_imbl(dat_use, 
                 var_, 
                 outcome,
                 key, 
                 scoring='balanced_accuracy', 
                 output_prefix = 'cv_tune', 
                 n_jobs=1, 
                 random_state=123, 
                 folder_out='/results/', 
                 kf=5, 
                 kf_inner=5):

    print(key)
    
    clf = lst_clf[key]
    
    data_X = dat_use[var_].values
    data_y = dat_use[outcome].values
    
    eval_metrics = pd.DataFrame()

    n = 0
    for train_index, test_index in kf.split(data_X, data_y):
        n = n+1
        print('fold_'+str(n))

        # training
        X_train = data_X[train_index]
        y_train = data_y[train_index]

        # X_train = np.concatenate((X_train, X_train_))
        # y_train = np.append(y_train, y_train_)

        print(X_train.shape)
        print(Counter(y_train))

        # test
        X_test = data_X[test_index]
        y_test = data_y[test_index]
        
        print(X_test.shape)
        print(Counter(y_test))


        pipeline = Pipeline([('scale', PowerTransformer()), #####
                                ("smote", RandomOverSampler(random_state=random_state)), 
                              ("clf", clf)])

        grid = lst_grid[key]

        # search = RandomizedSearchCV(
        #     pipeline, grid, 
        #     # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
        #     scoring=scoring,
        #     n_iter=100,random_state=random_state,
        #     n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        # ).fit(X_train, y_train)

        search = GridSearchCV(
            pipeline, grid, 
            # scoring=metrics.make_scorer(metrics.matthews_corrcoef),
            scoring=scoring, 
            n_jobs=n_jobs, cv=kf_inner,return_train_score=True
        ).fit(X_train, y_train)

        best_score = search.best_score_
        # print(f"Best Tuning balanced accuracy: {best_score}") ###############

        best_params = {
            key: value for key, value in search.best_params_.items()
        }
        # print(best_params)

        best_estimator = search.best_estimator_

        df_proba = pd.DataFrame()

        ave = 'macro'

        # y_pred_cv = np.append(y_pred_cv, y_pred)

        # area under ROC curve
        y_probs = best_estimator.predict_proba(X_test)[:,1]
        # y_prob_cv = np.append(y_prob_cv, y_probs)

        fp_rate, tp_rate, thresholds = metrics.roc_curve(y_test, y_probs)
        auc = metrics.auc(fp_rate, tp_rate)
        # auc_weight = metrics.roc_auc_score(y_test, y_pred_weight)

        y_pred = best_estimator.predict(X_test)

        # # probability
        # df_proba_ = pd.DataFrame()
        # df_proba_['proba'] = y_probs
        # df_proba_['true'] = y_test
        # df_proba = df_proba.append(df_proba_, ignore_index=True)

        # area under precision-recall curve
        precision_, recall_, thresholds_ = metrics.precision_recall_curve(y_test, y_probs)
        auc_pr = metrics.auc(recall_, precision_)

        precision = metrics.precision_score(y_test, y_pred, average=ave)
        precision_1 = metrics.precision_score(y_test, y_pred)
        precision_0 = metrics.precision_score(y_test, y_pred, pos_label=0)
        f1 = metrics.f1_score(y_test, y_pred, average=ave)
        f1_1 = metrics.f1_score(y_test, y_pred)
        f1_0 = metrics.f1_score(y_test, y_pred, pos_label=0)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        mcc = metrics.matthews_corrcoef(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred, average=ave)
        recall_1 = metrics.recall_score(y_test, y_pred)
        recall_0 = metrics.recall_score(y_test, y_pred, pos_label=0)
        balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

        brier = metrics.brier_score_loss(y_test, y_probs)

        eval_metrics = pd.concat([eval_metrics,pd.DataFrame.from_dict({'auroc': auc,
                                            'sensitivity': recall_1,
                                            'specificity': recall_0,
                                            'balanced accuracy': balanced_accuracy,
                                            'brier': brier}, orient='index').transpose()], ignore_index=True)    
    file_out = output_prefix+key+'.csv' ###############
    eval_metrics.to_csv(os.path.join(folder_out,file_out), index=False)
    
    return eval_metrics

    # y_test_pred = pd.DataFrame(np.concatenate((y_test_cv.reshape((-1,1)), y_pred_cv.reshape((-1,1))), axis=1))
    # y_test_pred.columns = ['test','pred']
    # print(y_test_pred.shape)
    # pred_out = 'y_test_pred_cv5_'+key+'.csv' ###################
    # y_test_pred.to_csv(os.path.join(folder_out, pred_out), index=False)
    
    